# import

In [1]:
import os
import pandas as pd


# Step 1: Read and filter the original files

In [17]:

# Step 1: Read and filter the original files
dfl_file_path = 'Data/dfl_traffic_count'
years_to_filter = [2018, 2019, 2021, 2022, 2023, 2024]

# List all files in the directory
file_paths = [os.path.join(dfl_file_path, file) for file in os.listdir(dfl_file_path) if file.endswith('.csv')]

for file_path in file_paths:
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Check if 'count_date' column exists
    if 'count_date' not in df.columns:
        print(f"'count_date' column not found in {file_path}. Skipping this file.")
        continue
    
    # Convert 'count_date' column to datetime type
    df['count_date'] = pd.to_datetime(df['count_date'], errors='coerce')
    
    # Filter rows for the specified years
    filtered_df = df[df['count_date'].dt.year.isin(years_to_filter)]
    
    # Construct new file name for the filtered data
    new_file_path = file_path.replace('.csv', '_filtered.csv')
    
    # Save the filtered data to a new file
    filtered_df.to_csv(new_file_path, index=False)
    print(f"Filtered data saved to {new_file_path}")


Filtered data saved to Data/dfl_traffic_count\dft_rawcount_local_authority_id_103_filtered.csv
Filtered data saved to Data/dfl_traffic_count\dft_rawcount_local_authority_id_107_filtered.csv


C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_8464\3120397016.py:10: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Filtered data saved to Data/dfl_traffic_count\dft_rawcount_local_authority_id_109_filtered.csv
Filtered data saved to Data/dfl_traffic_count\dft_rawcount_local_authority_id_110_filtered.csv
Filtered data saved to Data/dfl_traffic_count\dft_rawcount_local_authority_id_145_filtered.csv
Filtered data saved to Data/dfl_traffic_count\dft_rawcount_local_authority_id_174_filtered.csv
Filtered data saved to Data/dfl_traffic_count\dft_rawcount_local_authority_id_96_filtered.csv


# Step 2: Processing filtered files

In [18]:
# Step 2: Processing filtered files

for file_path in file_paths:
    # Construct the filtered file path
    filtered_file_path = file_path.replace('.csv', '_filtered.csv')
    
    # Read the filtered CSV file
    df = pd.read_csv(filtered_file_path)
    
    # Convert 'count_date' to datetime and extract the date part
    df['count_date'] = pd.to_datetime(df['count_date'])
    df['date'] = df['count_date'].dt.date
    
    # Group by day and calculate the mean
    daily_df = df.groupby('date').mean().reset_index()
    
    # Create a complete date range
    full_date_range = pd.date_range(start=daily_df['date'].min(), end=daily_df['date'].max())
    
    # Reindex to have a continuous date range
    daily_df = daily_df.set_index('date').reindex(full_date_range).reset_index()
    daily_df.rename(columns={'index': 'date'}, inplace=True)
    
    # Interpolate missing values using spline interpolation
    daily_df.interpolate(method='spline', order=3, inplace=True)
    
    # Save the processed data to a new CSV file
    processed_file_path = file_path.replace('.csv', '_processed.csv')
    daily_df.to_csv(processed_file_path, index=False)
    
    print(f"Processed data saved to {processed_file_path}")


C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_8464\3383438608.py:15: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  daily_df = df.groupby('date').mean().reset_index()


ValueError: Invalid fill method. Expecting pad (ffill) or backfill (bfill). Got spline

# filter

In [13]:
# Function to process each file
def process_file(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path, encoding='ISO-8859-1')  # Use ISO-8859-1 encoding to handle special characters
    
    # Convert 'count_date' to datetime type
    df['count_date'] = pd.to_datetime(df['count_date'], dayfirst=True, errors='coerce')
    
    # Filter rows based on the specified years
    filtered_df = df[df['count_date'].dt.year.isin(years_to_filter)]
    
    # Save the filtered data to a new file
    new_file_path = file_path.replace('.csv', '_filtered.csv')
    filtered_df.to_csv(new_file_path, index=False)
    print(f"Filtered data saved to {new_file_path}")
    
    # Convert 'count_date' to datetime and extract date part
    filtered_df['date'] = filtered_df['count_date'].dt.date
    
    # Group by day and calculate the mean
    daily_df = filtered_df.groupby('date').mean().reset_index()
    
    # Create a complete date range
    full_date_range = pd.date_range(start=daily_df['date'].min(), end=daily_df['date'].max())
    
    # Reindex to have a continuous date range
    daily_df = daily_df.set_index('date').reindex(full_date_range).reset_index()
    daily_df.rename(columns={'index': 'date'}, inplace=True)
    
    # Interpolate missing values for numerical columns
    numeric_cols = daily_df.select_dtypes(include='number').columns
    daily_df[numeric_cols] = daily_df[numeric_cols].interpolate(method='linear', limit_direction='both')
    
    # Save the processed data to a new CSV file
    output_file_path = new_file_path.replace('_filtered.csv', '_daily.csv')
    daily_df.to_csv(output_file_path, index=False)
    print(f"Processed data saved to {output_file_path}")

# Get all CSV files in the input folder
file_paths = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith('.csv')]

# Process each file
for file_path in file_paths:
    process_file(file_path)


Filtered data saved to Data/dfl_traffic_count\dft_rawcount_local_authority_id_103_filtered.csv


C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_8464\2640638767.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['date'] = filtered_df['count_date'].dt.date
C:\Users\ucfnxch\AppData\Local\Temp\ipykernel_8464\2640638767.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  daily_df = filtered_df.groupby('date').mean().reset_index()


Processed data saved to Data/dfl_traffic_count\dft_rawcount_local_authority_id_103_daily.csv


KeyError: 'count_date'

# delete

In [3]:
dfl_file_path = 'Data\dfl_traffic_count'

dft93 = 'Data\\traffic_count\dft_rawcount_local_authority_id_93.csv'
dft96 = 'Data\\traffic_count\dft_rawcount_local_authority_id_96.csv'
dft109 = 'Data\\traffic_count\dft_rawcount_local_authority_id_109.csv'
dft110 = 'Data\\traffic_count\dft_rawcount_local_authority_id_110.csv'
dft145 = 'Data\\traffic_count\dft_rawcount_local_authority_id_145.csv'
dft174 = 'Data\\traffic_count\dft_rawcount_local_authority_id_174.csv'


In [2]:


# 定义文件路径列表
file_paths = [
    'Data\\traffic_count\\dft_rawcount_local_authority_id_93.csv',
    'Data\\traffic_count\\dft_rawcount_local_authority_id_96.csv',
    'Data\\traffic_count\\dft_rawcount_local_authority_id_109.csv',
    'Data\\traffic_count\\dft_rawcount_local_authority_id_110.csv',
    'Data\\traffic_count\\dft_rawcount_local_authority_id_145.csv',
    'Data\\traffic_count\\dft_rawcount_local_authority_id_174.csv'
]

# 指定需要过滤的年份
years_to_filter = [2018, 2019, 2021, 2022, 2023, 2024]

for file_path in file_paths:
    # 读取CSV文件
    df = pd.read_csv(file_path)
    
    # 将count_date列转换为datetime类型
    df['count_date'] = pd.to_datetime(df['count_date'])
    
    # 过滤出指定年份的行
    filtered_df = df[df['count_date'].dt.year.isin(years_to_filter)]
    
    # 构建新的文件名
    new_file_path = file_path.replace('.csv', '_filtered.csv')
    
    # 保存过滤后的数据到新文件
    filtered_df.to_csv(new_file_path, index=False)

    print(f"Filtered data saved to {new_file_path}")

Filtered data saved to Data\traffic_count\dft_rawcount_local_authority_id_93_filtered.csv
Filtered data saved to Data\traffic_count\dft_rawcount_local_authority_id_96_filtered.csv


C:\Users\Chailee\AppData\Local\Temp\ipykernel_27736\327191506.py:18: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Filtered data saved to Data\traffic_count\dft_rawcount_local_authority_id_109_filtered.csv
Filtered data saved to Data\traffic_count\dft_rawcount_local_authority_id_110_filtered.csv
Filtered data saved to Data\traffic_count\dft_rawcount_local_authority_id_145_filtered.csv
Filtered data saved to Data\traffic_count\dft_rawcount_local_authority_id_174_filtered.csv
